In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Tích hợp dữ liệu/preprocessed data'

/content/drive/.shortcut-targets-by-id/1kdSQxLU563_8sd8nvQTr-Z08SiwQ3jyp/Tích hợp dữ liệu/preprocessed data


In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy,datetime,re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


#Schema matching

In [ ]:
#READ DATA:
IMDB_DF = pd.read_csv("dmx.csv")
IMDB_DF = IMDB_DF[["ten", "gia", "loai_may", "hang", "khoi_luong_giat"]]
IMDB_DF_SAMPLE = IMDB_DF.sample(10).astype(str)
imdb_header_list = list(IMDB_DF.columns.values)

#imdb_header
imdb_header_array = np.array(imdb_header_list)

In [ ]:
#ROTTEN_TOMATOES_DF = pd.read_csv("dmcl.csv")
#ROTTEN_TOMATOES_DF = ROTTEN_TOMATOES_DF[[ "ten", "gia", "loai_may_cua", "hang", "khoi_luong_giat"]]
#ROTTEN_TOMATOES_DF = pd.read_csv("nk.csv")
#ROTTEN_TOMATOES_DF = ROTTEN_TOMATOES_DF[["title", "price", "Loại máy giặt:", "Nhà sản xuất:", "khoi_luong_giat"]]
ROTTEN_TOMATOES_DF = pd.read_csv("pico.csv")
ROTTEN_TOMATOES_DF = ROTTEN_TOMATOES_DF[["ten", "gia", "loai_may", "hang", "khoi_luong_giat"]]
ROTTEN_TOMATOES_DF_SAMPLE = ROTTEN_TOMATOES_DF.sample(10).astype(str)
rt_header_list = list(ROTTEN_TOMATOES_DF.columns.values)
#rt_header
rt_header_array = np.array(rt_header_list)

In [ ]:
#creation array to add sort index of matching
# For matching
Pref=np.zeros((len(imdb_header_array),len(rt_header_array)))
#for Tomatoes
pref_tom=np.zeros((len(imdb_header_array),len(rt_header_array)),dtype=np.int8)
ipref_tom=np.zeros((len(imdb_header_array),len(rt_header_array)),dtype=np.int8)
#for IMdb
ipref_imdb=np.zeros((len(rt_header_array),len(imdb_header_array)),dtype=np.int8)
pref_imdb=np.zeros((len(rt_header_array),len(imdb_header_array)),dtype=np.int8)

In [ ]:
def check_short_matching(COL1,COL2):
    rate = 0.0
    if str(COL1) == 'nan' or str(COL2) == 'nan' :
        return 0
    COL1 = str(COL1)
    COL2 = str(COL2)
    sw = (fuzz.SequenceMatcher(None,COL1, COL2).ratio())
    return sw #/(max(len(COL1),len(COL2))*1.0)

In [ ]:
def check_match(st1,st2):
    str1 = np.array((st1.values).astype(str))
    str2 = np.array((st2.values).astype(str))
    ret = np.zeros((len(str1),len(str2)))
    for i in range(len(str1)):
        for j in range(len(str2)):
            ret[i,j] = check_short_matching(str1[i],str2[j])
    return np.average(ret)

In [ ]:
index_i = 0
for i in IMDB_DF_SAMPLE:
    index_j = 0
    for j in ROTTEN_TOMATOES_DF_SAMPLE:
        table1 = IMDB_DF_SAMPLE[i]
        table2 = ROTTEN_TOMATOES_DF_SAMPLE[j]
        Pref[index_i,index_j] = 100 * (np.float(check_match(table1, table2)))
        index_j+=1
    index_i+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [ ]:
#assign the array indexs of Pref sorted by the actual values decending based on the lenght of rt_header_array (17)
for i in range(len(rt_header_array)):
    ipref_tom[:,i]=np.argsort(Pref[:,i])
    ipref_tom[:,i]=ipref_tom[::-1,i]
    pref_tom[:,i]=np.sort(Pref[:,i])
    pref_tom[:,i]=pref_tom[::-1,i]
#ipref_tom

In [ ]:
#assign the array indexs of Pref sorted by the actual values decending based on the lenght of imdb_header_array (13)
for i in range(len(imdb_header_array)):
    #get indexs values of rows and store them in the rows as well
    ipref_imdb[:,i] = np.argsort(Pref[i,:])
    ipref_imdb[:,i] = ipref_imdb[::-1,i]
    pref_imdb[:,i] = np.sort(Pref[i,:])

In [ ]:
#create list of preferred_rankings_tomatoes with header
preferred_rankings_tomatoes = {}
for i in range(len(rt_header_array)):
    preferred_rankings_tomatoes[i] = ipref_tom[:,i]


In [ ]:
#create list of preferred_rankings_Imdb with header
preferred_rankings_imdb = {}
for i in range(len(imdb_header_array)):
    preferred_rankings_imdb[i] = ipref_imdb[:,i]

In [ ]:
tentative_engagements = []
free_imdbs = []
corresponacy1 = []

In [ ]:
#Imdb which still need to propose and get accepted successfully
#Keep track of the headers that "may" end up together
def init_free_imdbs():
    #corresponacy1 = list.clear()
    #'''Initialize the arrays of tomates and imdbs to represent that they're all initially free and not engaged'''
    for imdb in preferred_rankings_imdb.keys():
        free_imdbs.append(imdb)

In [ ]:
def begin_matching(imdb):
    #Find the first free Tomaten available to a Imdb at any given time
    for tomate in preferred_rankings_imdb[imdb]:
        #Boolean for whether tomate is taken or not
        taken_match = [couple for couple in tentative_engagements if tomate == couple [:][1]]
        if (len(taken_match) == 0):
            #tentatively engage the man and woman
            tentative_engagements.append([imdb, tomate])
            free_imdbs.remove(imdb)
            break
        elif (len(taken_match) > 0):
            current_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(taken_match[0][0])
            potential_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(imdb)
            if (current_imdb < potential_imdb):
                ii=[]
            else: 
                #The new imdb is engaged
                free_imdbs.remove(imdb)
                #The old imdb is now single
                free_imdbs.append(taken_match[0][0])
                #Update the fiance of the tomaten (tentatively)
                taken_match[0][0] = imdb
                break

In [ ]:
def stable_matching():
    '''Matching algorithm until stable match terminates'''
    while (len(free_imdbs) > 0):
        for imdb in free_imdbs:
            begin_matching(imdb)

In [ ]:
def output():
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        if(matching > 10.0):
            corresponacy1.append(item)
            print(f'{imdb_header_array[item[0]]} ----- {rt_header_array[item[1]]} with score {matching}')

#Result

In [ ]:
#dmx vs pico
def main():
    tentative_engagements.clear()
    free_imdbs.clear()
    corresponacy1.clear()
    init_free_imdbs()

    print(free_imdbs)
    stable_matching()
    print(tentative_engagements)
    output()
main()

[0, 1, 2, 3, 4]
[[0, 0], [2, 2], [4, 4], [1, 1], [3, 3]]
ten ----- ten with score 39.496554178774915
loai_may ----- loai_may with score 50.761107742989054
khoi_luong_giat ----- khoi_luong_giat with score 26.563492063492067
gia ----- gia with score 67.77037455298324
hang ----- hang with score 35.44156900890647


In [ ]:
#dmx vs dmcl
def main():
    tentative_engagements.clear()
    free_imdbs.clear()
    corresponacy1.clear()
    init_free_imdbs()

    print(free_imdbs)
    stable_matching()
    print(tentative_engagements)
    output()
main()

[0, 1, 2, 3, 4]
[[0, 0], [2, 2], [4, 4], [1, 1], [3, 3]]
ten ----- ten with score 50.82955369801915
loai_may ----- loai_may_cua with score 76.13643790849673
khoi_luong_giat ----- khoi_luong_giat with score 28.004761904761903
gia ----- gia with score 69.25108225108227
hang ----- hang with score 29.50023541989486


In [ ]:
#dmx vs nk
def main():
    tentative_engagements.clear()
    free_imdbs.clear()
    corresponacy1.clear()
    init_free_imdbs()

    print(free_imdbs)
    stable_matching()
    print(tentative_engagements)
    output()
main()

[0, 1, 2, 3, 4]
[[0, 0], [2, 2], [4, 4], [1, 1], [3, 3]]
ten ----- title with score 61.11125128257977
loai_may ----- Loại máy giặt: with score 56.84338432287615
khoi_luong_giat ----- khoi_luong_giat with score 21.764285714285712
gia ----- price with score 66.02973837756448
hang ----- Nhà sản xuất: with score 23.311929642425
